<a href="https://colab.research.google.com/github/AlAntonov/Mari/blob/main/Mari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("AigizK/mari-russian-parallel-corpora")
shf_dataset = dataset.shuffle(seed=42)
split_dataset = shf_dataset['train'].train_test_split(test_size=0.01)

In [ ]:
!mkdir '/content/data'
!mkdir '/content/data/train'
!mkdir '/content/data/dev'

In [ ]:
def writelines(list, filename):
  with open(filename, 'w', encoding='utf-8') as file:
    for item in list:
      file.write('%s\n' % item)

mhr_train = [l['mhr'] for l in split_dataset['train']]
rus_train = [l['rus'] for l in split_dataset['train']]
mhr_dev = [l['mhr'] for l in split_dataset['test']]
rus_dev = [l['rus'] for l in split_dataset['test']]

writelines(mhr_train, '/content/data/train/train.mhr.txt')
writelines(rus_train, '/content/data/train/train.rus.txt')
writelines(mhr_dev, '/content/data/dev/dev.mhr.txt')
writelines(rus_dev, '/content/data/dev/dev.rus.txt')

In [ ]:
pip install sockeye

In [ ]:
!git clone https://github.com/rsennrich/subword-nmt.git

In [ ]:
src = 'mhr'
tgt = 'rus'
name = '355K'

In [ ]:
!python subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input /content/data/train/train.{src}.txt /content/data/train/train.{tgt}.txt -s 10000 -o /content/data/bpe.codes --write-vocabulary /content/data/bpe.vocab.{src} /content/data/bpe.vocab.{tgt}

In [ ]:
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{src} --vocabulary-threshold 50 < /content/data/train/train.{src}.txt > /content/data/train/train.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < /content/data/train/train.{tgt}.txt > /content/data/train/train.{tgt}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{src} --vocabulary-threshold 50 < /content/data/dev/dev.{src}.txt > /content/data/dev/dev.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < /content/data/dev/dev.{tgt}.txt > /content/data/dev/dev.{tgt}.bpe


In [ ]:
!python -m sockeye.prepare_data -s /content/data/train/train.{src}.bpe -t /content/data/train/train.{tgt}.bpe --shared-vocab -o /content/{src}_{tgt}_{name}_data

In [ ]:
!python -m sockeye.train -d /content/{src}_{tgt}_{name}_data -vs /content/data/dev/dev.{src}.bpe -vt /content/data/dev/dev.{tgt}.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o {src}_{tgt}_{name}_model --num-layers 6 --keep-last-params 3 --batch-size 1024 --optimized-metric bleu --max-num-checkpoint-not-improved 7